<a href="https://colab.research.google.com/github/HedersonSantos/age-related/blob/main/auto_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#!pip install h2o

In [14]:
#!wget https://raw.githubusercontent.com/HedersonSantos/age-related/main/tree_features\ .csv
#!mv tree_features\ .csv tree_features.csv

In [2]:
import pandas as pd


In [4]:
df = pd.read_csv('tree_features.csv', header=0)
df['Class']=df['Class'].apply(lambda x: 'S' if x==1 else 'N')
df['Class']=df['Class'].astype('category')
df.head(10)

,AB,AF,BC,BQ,CC,CR,DA,DE,DH,DI,DU,EE,EH,FD,FE,FI,FL,FR,GL,Class
0,0.209377,3109.03329,5.555634,152.707705,0.563481,0.069225,69.08340,295.570575,0.284232,89.245560,5.310690,1.987283,0.949104,10.265073,9028.291921,3.583450,7.298162,1.73855,0.120343,S
1,0.145282,978.76416,1.229900,14.754720,0.484710,1.117800,70.79836,178.553100,0.363489,110.581815,0.005518,0.858603,0.003042,0.296850,6785.003474,10.358927,0.173229,0.49706,21.978000,N
2,0.470030,2635.10654,1.229900,219.320160,0.495852,0.700350,70.81970,321.426625,0.210441,120.056438,1.289739,8.146651,0.377208,8.745201,8338.906181,11.626917,7.709560,0.97556,0.196941,N
3,0.252107,3819.65177,1.229900,11.050410,0.717882,0.636075,47.27586,196.607985,0.292431,139.824570,2.655345,3.813326,0.614484,7.884336,10965.766040,14.852022,6.122162,0.49706,0.155829,N
4,0.380297,3733.04844,102.151980,149.717165,0.536467,0.693150,74.06532,200.178160,0.207708,97.920120,1.144902,3.490846,0.164268,4.274640,16198.049590,13.666727,8.153058,48.50134,0.096614,S
5,0.209377,2615.81430,1.229900,16.526120,0.639460,0.857625,55.22404,135.489250,0.478275,135.317865,0.005518,2.394414,0.003042,0.296850,8517.278846,10.981896,0.173229,0.49706,21.978000,N
6,0.348249,1733.65412,1.229900,344.644105,0.946323,0.610950,19.21570,107.907985,0.460510,176.625563,1.006962,2.620150,0.097344,1.460502,3903.806766,10.777915,4.408484,0.86130,0.065096,N
7,0.269199,966.45483,1.229900,6.199900,0.771909,1.109625,63.21684,326.225295,0.325227,83.769368,2.117379,3.277203,0.292032,4.518057,18090.349450,10.342388,6.591896,0.49706,0.092873,N
8,0.346113,3238.43674,3.626448,61.642115,0.855496,1.050375,76.77356,231.134460,0.330693,131.349555,0.613833,1.342323,0.066924,1.389258,3380.026318,11.450501,4.762291,1.18262,0.073416,N
9,0.324748,5188.68207,1.229900,107.276505,0.718271,0.711712,31.17774,403.247940,0.351191,129.175267,0.005518,3.658132,0.003042,0.296850,3142.390734,12.329825,0.173229,1.57151,21.978000,N


In [17]:
X_train = df[:-1].values
y_train = df['Class']


In [18]:
import h2o
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.automl import H2OAutoML


In [19]:
h2o.init()
health = h2o.H2OFrame(df)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 min 47 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,25 days
H2O_cluster_name:,H2O_from_python_unknownUser_yc3qch
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.169 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [20]:
# set the predictor names and the response column name
predictors = ['AB', 'AF', 'BC', 'BQ', 'CC', 'CR',
              'DA', 'DE', 'DH', 'DI', 'DU', 'EE',
              'EH', 'FD ', 'FE', 'FI', 'FL', 'FR', 'GL']
response = "Class"

# split into train and validation sets
train, valid = health.split_frame(ratios = [.90], seed = 1234)



In [21]:
aml = H2OAutoML(max_runtime_secs=60*120,
                seed=1,
                balance_classes=False,
                project_name='age_related'
)
%time aml.train(x=predictors, y=response, training_frame=train)

AutoML progress: |
23:58:24.656: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.
23:58:29.300: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.
23:58:31.181: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.
00:00:04.62: New models will be added to existing leaderboard age_related@@Class (leaderboard frame=null) with already 2 models.

████
00:00:16.237: StackedEnsemble_BestOfFamily_1_AutoML_2_20230524_00004 [StackedEnsemble best_of_family_1 (built with AUTO metalearner, using top model f

OSError: ignored